ffmpeg - https://www.gyan.dev/ffmpeg/builds/

Ubuntu: sudo apt install ffmpeg  
在 Windows 系统上安装用于视频处理的 ffmpeg、opencv-python 和 moviepy 包的步骤如下：

## 安装 ffmpeg 

下载 ffmpeg：
访问 ffmpeg 官方下载页面。 找到 Windows 版本的 ffmpeg，并下载适合你系统的压缩包（通常是 ffmpeg-release-essentials.zip）。 

解压并配置 ffmpeg：
将下载的压缩包解压到一个你方便访问的文件夹（例如 C:\ffmpeg）。 确保文件夹结构如下：C:\ffmpeg\bin\ffmpeg.exe。 添加 ffmpeg 到系统路径：

右键点击“此电脑”或“计算机”，选择“属性”。 选择“高级系统设置”。 在“系统属性”窗口中，点击“环境变量”。 在“系统变量”部分，找到并选中“Path”，然后点击“编辑”。 点击“新建”，然后输入 C:\ffmpeg\bin，点击“确定”。 关闭所有窗口，并重启命令提示符以应用更改。 


## 安装 opencv-python 和 moviepy 

打开命令提示符（可以通过按 Win + R 然后输入 cmd 打开），然后执行以下命令来安装 opencv-python 和 moviepy：

pip install opencv-python --quiet 
pip install moviepy --quiet 

这些命令将会安装最新版本的 opencv-python 和 moviepy 包，并且使用 --quiet 标志来抑制安装过程中的详细输出。

验证安装 在安装完成后，你可以通过以下 Python 脚本来验证安装是否成功：

import cv2 import moviepy.editor as mp

print("OpenCV version:", cv2.version) print("MoviePy version:", mp.version) 

运行以上脚本，如果没有错误，并且打印出了 OpenCV 和 MoviePy 的版本号，说明安装成功。

通过这些步骤，你应该能够在 Windows 系统上成功安装并配置用于视频处理的 ffmpeg、opencv-python 和 moviepy 包。


In [1]:
# %pip install opencv-python --quiet
# %pip install moviepy --quiet

In [3]:
pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.3/54.8 MB ? eta -:--:--0m

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pip/_vendor/urllib3/response.py", line 438, in _error_catcher
    yield
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pip/_vendor/urllib3/response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ~~~~~~~~~~~~~^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pip/_vendor/urllib3/response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ~~~~~~~~~~~~~^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pip/_vendor/cachecontrol/filewrapper.py", lin

In [2]:
# We'll be using the OpenAI DevDay Keynote Recap video. You can review the video here: https://www.youtube.com/watch?v=h02ti0Bl6zk
MODEL = 'gpt-4o'

导入必要的模块，包括os用于路径操作，cv2用于视频处理，base64用于编码帧，以及moviepy.editor用于处理音频。函数extract_frames_and_audio，用于提取视频帧和音频。参数video_file为视频文件路径，interval为每提取一帧所间隔的秒数。使用OpenCV遍历视频并提取指定间隔的帧。将每个帧编码为Base64格式并存储在列表中。使用MoviePy从视频中提取音频并保存为MP3文件。

In [2]:
# 导入所需的库
import os
import cv2 # 视频处理
import base64 # 编码帧
from moviepy.editor import VideoFileClip # 音频处理

VIDEO_FILE = "Good_Driver.mp4"

def extract_frames_and_audio(video_file, interval=2):
    encoded_frames = []
    file_name, _ = os.path.splitext(video_file)

    video_capture = cv2.VideoCapture(video_file)
    total_frame_count = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = video_capture.get(cv2.CAP_PROP_FPS)
    frames_interval = int(frame_rate * interval)
    current_frame = 0

    # 循环遍历视频并以指定的采样率提取帧
    while current_frame < total_frame_count - 1:
        video_capture.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        success, frame = video_capture.read()
        if not success:
            break
        _, buffer = cv2.imencode(".jpg", frame)
        encoded_frames.append(base64.b64encode(buffer).decode("utf-8"))
        current_frame += frames_interval
    video_capture.release()

    # 从视频中提取音频
    audio_output = f"{file_name}.mp3"
    video_clip = VideoFileClip(video_file)
    video_clip.audio.write_audiofile(audio_output, bitrate="32k")
    video_clip.audio.close()
    video_clip.close()

    print(f"提取了 {len(encoded_frames)} 帧")
    print(f"音频提取到 {audio_output}")
    return encoded_frames, audio_output

# 每2秒提取1帧（采样率）
encoded_frames, audio_output = extract_frames_and_audio(VIDEO_FILE, interval=2)

ModuleNotFoundError: No module named 'cv2'

In [4]:
import time
from IPython.display import Image, display, Audio

# 创建显示句柄以动态更新显示内容
display_handle = display(None, display_id=True)

# 显示提取的帧，每帧之间暂停0.025秒
for frame in encoded_frames:
    display_handle.update(Image(data=base64.b64decode(frame.encode("utf-8")), width=600))
    time.sleep(0.025)

# 显示提取的音频
Audio(audio_output)

In [5]:
# 创建OpenAI客户端
from openai import OpenAI 
client = OpenAI()

# 使用GPT-4o模型生成视频介绍
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {"role": "system", "content": "请用Markdown格式生成视频的介绍."},
        {"role": "user", "content": [
            "下面是视频的图像帧",
            *map(lambda x: {"type": "image_url", "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}}, encoded_frames)
        ]},
    ],
    temperature=0,
)

# 打印生成的Markdown格式介绍
print(response.choices[0].message.content)

NameError: name 'client' is not defined

In [ ]:
# 使用GPT-4o模型根据视频内容回答问题

QUESTION = "图中的人做了什么?"

qa_response = client.chat.completions.create(
    model=MODEL,
    messages=[
    {"role": "system", "content": "请用Markdown格式根据视频内容回答问题."},
    {"role": "user", "content": [
        "下面是视频的图像帧.",
        *map(lambda x: {"type": "image_url", "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "low"}}, encoded_frames),
        QUESTION
        ],
    }
    ],
    temperature=0,
)

# 打印生成的Markdown格式问题回答
print(QUESTION + "\n" + qa_response.choices[0].message.content)